# Eclerx (Hard Level) - PySpark Interview Question

You are working as a Data Engineer and need to clean up a dataset that contains customer order information. The dataset includes details such as the customer ID, order ID, order date, and the total amount. Due to a data processing issue, some rows are duplicated, and you need to remove duplicates based on a composite key of customer_id and order_id, keeping only the latest order (based on the order_date).

You need to remove the duplicate rows based on the composite key (customer_id, order_id) and retain only the row with the latest order_date for each combination of customer_id and order_id.


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [0]:
data = [ (101, 1001, "2025-01-15", 500.00), (102, 1002, "2025-01-14", 300.00), (101, 1001, "2025-01-17", 550.00), (103, 1003, "2025-01-16", 450.00), 
(102, 1002, "2025-01-18", 320.00), (103, 1003, "2025-01-19", 460.00) ] 

# Define schema 
schema = ["customer_id", "order_id", "order_date", "total_amount"]

df = spark.createDataFrame(data, schema)
df.display()

customer_id,order_id,order_date,total_amount
101,1001,2025-01-15,500.0
102,1002,2025-01-14,300.0
101,1001,2025-01-17,550.0
103,1003,2025-01-16,450.0
102,1002,2025-01-18,320.0
103,1003,2025-01-19,460.0


In [0]:
window_criteria = Window.partitionBy(col('composite_id')).orderBy(col('order_date').desc())

(
    df.withColumn('composite_id', concat(col('customer_id'), col('order_id')))
        .withColumn( 'rnk', rank().over(window_criteria))
        .filter(col('rnk') == 1)
        .drop('rnk' , 'composite_id')
    
    .display()
)

customer_id,order_id,order_date,total_amount
101,1001,2025-01-17,550.0
102,1002,2025-01-18,320.0
103,1003,2025-01-19,460.0
